# River Mouth Water Quality Data
Collection of river mouth water quality data from Copernicus Marine Service products, starting with utilization of CMEMS WMS for data retrieval.  

_Author: Peter Kerins_  
_Created: 9 Jul 2021_  
_Environment: jupyterlab_  

Ocean Watch seeks to feature several biogeochemical parameters from the [Global Ocean Biogeochemistry Analysis and Forecast (GLOBAL_ANALYSIS_FORECAST_BIO_001_028) product](https://resources.marine.copernicus.eu/?option=com_csw&view=details&product_id=GLOBAL_ANALYSIS_FORECAST_BIO_001_028) of the [Copernicus Marine Service](https://marine.copernicus.eu/). The data will be mapped via Resource Watch. OW will also show time series of the parameters at various locations (representing outlets of rivers to the ocean).  

# Source

Presenting that time series data would typically require harvesting the information from the historical product rasters. However, the CMEMS [Pretty View](https://view-cmems.mercator-ocean.fr/GLOBAL_ANALYSIS_FORECAST_BIO_001_028) WebGIS tool displays just such time series (via the `Tools`). Furthermore, these are generated using the public-facing [WMS](https://www.ogc.org/standards/wms). The Copernicus Marine User Support Expert indicated [the relevant documentation](https://docs.geoserver.org/master/en/user/services/wms/reference.html#getfeatureinfo).

Constructing a properly formatted `GetFeatureInfo` request from scratch without examples was unsuccessful. However, spying on the Pretty View tool while making a time series request revealed [just such an example](https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-daily?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetFeatureInfo&QUERY_LAYERS=chl&BBOX=53.26,14.26,53.2600001,14.2600001&HEIGHT=1&WIDTH=1&INFO_FORMAT=text/xml&SRS=EPSG:4326&X=0&Y=0&elevation=-0.49402499198913574&time=2020-07-16T12:00:00.000Z/2021-07-16T12:00:00.000Z):

```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-daily?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=chl
&BBOX=53.26,14.26,53.2600001,14.2600001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2020-07-16T12:00:00.000Z/2021-07-16T12:00:00.000Z
```

This query returns daily readings of `mass_concentration_of_chlorophyll_a_in_sea_water` at a depth of about half a meter for approximately the preceding year, at a location between Socotra and Yemen. 

A similar example closer to OW's needs can also be generated. [The query](https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?SERVICE=WMS&VERSION=1.1.1&REQUEST=GetFeatureInfo&QUERY_LAYERS=o2&BBOX=31.01,31.74,31.010000100000003,31.7400001&HEIGHT=1&WIDTH=1&INFO_FORMAT=text/xml&SRS=EPSG:4326&X=0&Y=0&elevation=-0.49402499198913574&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z) for the full times series of monthly `mole_concentration_of_dissolved_molecular_oxygen_in_sea_water` values at a half-meter of depth at a pixel near the mouth of the Nile:
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=o2
&BBOX=31.01,31.74,31.010000100000003,31.7400001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z
```

This template can be modified to accommodate OW needs.

Capabilities of service requested via:
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetCapabilities
```

Key excerpt from response:
```xml
            <GetFeatureInfo>
                <Format>image/png</Format>
                <Format>text/xml</Format>
                <DCPType>
                    <HTTP>
                        <Get>
                            <OnlineResource xlink:type="simple" xlink:href="http://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly" />
                        </Get>
                    </HTTP>
                </DCPType>
            </GetFeatureInfo>
```
Data can only be returned in `text/xml` format, not the preferred JSON (`application/json`).

# Parameters

## Variable
- _mole_concentration_of_dissolved_molecular_oxygen_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=o2`  
- _mole_concentration_of_phosphate_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=no3`  
- _mole_concentration_of_nitrate_in_sea_water (mmol m-3)_  
  - `QUERY_LAYERS=po4`  
  
Note: providing a comma-separated list of valid layers (eg `QUERY_lAYERS=o2,no3`) returns an exception (`400` response)


## Depth
- `elevation=-0.49402499198913574`
- `elevation=-1.5413750410079956`
- `elevation=-2.6456689834594727`
- `elevation=-3.8194949626922607`
- `elevation=-5.078224182128906`

## Bounding Box
`xmin`,`ymin` are just the point of selection; `xmax`, `ymax` are just minimum value plus 0.0000001

# Example Request

Oxygen
```html
https://nrt.cmems-du.eu/thredds/wms/global-analysis-forecast-bio-001-028-monthly?
SERVICE=WMS
&VERSION=1.1.1
&REQUEST=GetFeatureInfo
&QUERY_LAYERS=o2
&BBOX=31.01,31.74,31.010000100000003,31.7400001
&HEIGHT=1
&WIDTH=1
&INFO_FORMAT=info_format=text/xml
&SRS=EPSG:4326
&X=0
&Y=0
&elevation=-0.49402499198913574
&time=2019-01-16T12:00:00.000Z/2021-05-16T12:00:00.000Z
```

# Workflow
Utilizing the CMEMS data within an OW widget entailed several steps. And prerequisite to all of those was a phase in which a river mouth dataset was constructed.

## Compile River Mouths
 - Identify river mouth locations
     - Script: [river-mouths_identify-points.js](./river-mouths_identify-points.js)
         - [On Google Earth Engine](https://code.earthengine.google.com/736a3ca06cf73e1d4a2894f4dc42b5f0)
     - Key source: [MERIT Hydro](https://doi.org/10.1029/2019WR024873)
         - Google Earth Engine asset `MERIT/Hydro/v1_0_1`
     - Set of river mouths stored as points
         - Google Earth Engine asset `projects/resource-watch-gee/ocean-watch/river-mouths`
 - Affiliate river mouths with actual rivers
     - Script: [river-mouths_match-mouths-with-rivers.js](./river-mouths_match-mouths-with-rivers.js)
         - [On Google Earth Engine](https://code.earthengine.google.com/fd7e922bb8a4e55770dc6f3f9159df9d)
     - Key source: [HydroRIVERS](https://www.hydrosheds.org/page/hydrorivers)
         - River network uploaded to Google Earth Engine as asset `projects/resource-watch-gee/ocean-watch/river-network`
     - Every MERIT Hydro river mouth associated with a HydroRIVERS river
         - Output manually uploaded to [Carto table](https://resourcewatch.carto.com/u/wri-rw/dataset/ocn_calcs_010_target_river_mouths)

## Label Rivers
 - Harvest river names
     - Script: [river-mouths_harvest-names.py](./river-mouths_harvest-names.py)
         - Result written to shapefile locally
     - Key source: [OpenStreetMap](https://www.openstreetmap.org/)
 - Collect names for missing areas
     - Notebook: [river-mouths_harvest-missing.ipynb](river-mouths_harvest-missing.ipynb)
         - Retrieved data merged with that from previous step and saved as shapefile locally
     - Key source: Again [OpenStreetMap](https://www.openstreetmap.org/)
 - Assign names to river mouths
     - Notebook: [river-mouths_assign-names.ipynb](river-mouths_assign-names.ipynb)
         - Describes manual execution in ArcMap, the output of which is passed to the Carto table containing the completed river mouth point dataset
         - Not all rivers identified in the MERIT Hydro/HydroRIVERS dataset are named in OSM data

## Collect Water Quality Data
 - Identify coordinates for each river mouth that overlap with the ocean water quality data products
     - Script: [river-mouths_adjust-coordinates.py](./river-mouths_adjust-coordinates.py)
     - This only needs to be executed once. The results are stored persistently on Carto.  
     - Valid coordinates could not be algorithmically identified for every river mouth, leaving about 500 outstanding river mouths. From these, valid coordinates were manually identified for the dozens of rivers with `ORD_FLOW` rank higher than 5, leaving over 400 rank 5 rivers without coordinates. These are to be addressed at a later date.
 - Test adjusted coordinates for validity (ie valid WMS response)
     - Script: [river-mouths_test-coordinates.py](./river-mouths_test-coordinates.py)
     - This identifies river mouths where the "valid" coordinates still do not return a proper response. These must then be manually corrected within the river mouths table.
     - Note that the WMS is not perfectly consistent, so coordinates may be successfully tested but still fail during data harvesting. This seems to occur when the adjusted coordinates are very near the edge of the land mask.
 - Add basin information to river mouth data
     - Script: [river-mouths_join-basins.py](./river-mouths_join-basins.py)
     - Joining `PFAF_ID` and `HYBAS_ID` from HydroBASINS dataset to the river mouths dataset, which is largely based on the HydroRIVERS dataset, links each river mouth to the containing basin at any desired levels (here 3 and 6).
 - For validated river mouth locations, use WMS calls to harvest and store all relevant time series data
     - Script: [river-mouths_data-collection.py](./river-mouths_data-collection.py)
     - These actual river mouth quality data are stored in [a separate table](https://resourcewatch.carto.com/u/wri-rw/dataset/ocn_020alt_chemical_concentrations).

### Performance Enhancement
Indices added to river basins table for faster service of data to parameterized chart widgets

```sql
CREATE INDEX ON wat_068_rw0_watersheds_edit
(
	geostore_prod  
)
WHERE level = 5 AND coast = 1
```

```sql
CREATE INDEX ON wat_068_rw0_watersheds_edit
(
	hybas_id  
)
WHERE level = 5 AND coast = 1
```

# Results

## River Mouth Locations
Original locations and effective (ie valid for WMS request) coordinates are stored in [the same table](https://resourcewatch.carto.com/u/wri-rw/dataset/ocn_calcs_010_target_river_mouths), with the initially calculated location represented by the geometry, and the validated coordinates stored in `x_valid` and `y_valid`. 

In [1]:
import os
from dotenv import dotenv_values
env_vars = dotenv_values('/home/pkerins/code/.env')
from cartoframes.auth import set_default_credentials
from cartoframes import read_carto, to_carto
import pandas as pd
CARTO_USER = env_vars['CARTO_WRI_RW_USER']
CARTO_KEY = env_vars['CARTO_WRI_RW_KEY']
set_default_credentials(username=CARTO_USER,
                        base_url="https://{user}.carto.com/".format(user=CARTO_USER),
                        api_key=CARTO_KEY)

In [2]:
read_carto('ocn_calcs_010_target_river_mouths').tail()

,cartodb_id,the_geom,main_riv,distance,upland_skm,ord_clas,ord_stra,catch_skm,length_km,dist_dn_km,...,hyriv_id,dis_av_cms,endorheic,hybas_l12,x_valid,y_valid,pfaf_id_12,hybas_id_5,hybas_id_3,osm_name
5724,3496,POINT (-43.60638 60.50050),90135368,23288.995625,167.3,1,3,7.97,7.62,0.0,...,90135368,10.235,0,9120006730,-43.30637859444319,60.50050341001672,912559380000,9050006250,9030005280,None
5725,3523,POINT (-43.60638 60.50050),90135387,23413.600335,426.5,1,4,2.94,1.65,0.0,...,90135387,27.240,0,9120006750,-43.30637859444319,60.50050341001672,912559401000,9050006250,9030005280,None
5726,3532,POINT (-43.60638 60.50050),90135880,47427.292277,200.3,1,4,2.32,0.98,0.0,...,90135880,10.212,0,9120007080,-43.30637859444319,60.50050341001672,912559580000,9050006250,9030005280,None
5727,3504,POINT (-45.08186 60.19005),90136552,21526.081185,160.1,1,3,1.70,0.98,0.0,...,90136552,11.853,0,9120007060,-45.15257013833865,60.260758095491326,912559560000,9050006250,9030005280,Kuussuaq
5728,3544,POINT (-43.60638 60.13507),90136771,5836.664738,152.9,1,4,1.71,2.42,0.0,...,90136771,13.799,0,9120006890,-43.606378594443186,59.83507116048474,912559540000,9050006250,9030005280,None


## Chemical Concentrations at River Mouths
Corrected locations are used to retrieve longitudinal data for the concentration of various chemicals at a range of depths. The results for all locations, chemicals, and depths are stored in [a single table](https://resourcewatch.carto.com/u/wri-rw/dataset/ocn_calcs_011_river_mouth_chemical_concentrations).  

In [3]:
read_carto('ocn_020alt_chemical_concentrations').tail()

,cartodb_id,the_geom,longitude,latitude,gridcentrelon,gridcentrelat,dt,variable,depth,value,hyriv_id,pfaf_id_12,hybas_id_5,hybas_id_3,ord_flow,osm_name
2554705,2554551,None,-43.606379,59.835071,-43.5,59.75,2021-03-16 12:00:00,o2,-0.494025,336.01984,90136771,912559540000,9050006250,9030005280,5,None
2554706,2554552,None,-43.606379,59.835071,-43.5,59.75,2021-04-16 00:00:00,o2,-0.494025,344.45526,90136771,912559540000,9050006250,9030005280,5,None
2554707,2554553,None,-43.606379,59.835071,-43.5,59.75,2021-05-16 12:00:00,o2,-0.494025,343.17358,90136771,912559540000,9050006250,9030005280,5,None
2554708,2554554,None,-43.606379,59.835071,-43.5,59.75,2021-06-16 00:00:00,o2,-0.494025,346.04340,90136771,912559540000,9050006250,9030005280,5,None
2554709,2554555,None,-43.606379,59.835071,-43.5,59.75,2021-07-16 12:00:00,o2,-0.494025,342.50235,90136771,912559540000,9050006250,9030005280,5,None
